In [1]:
import tensorflow as tf

# Checks for available GPUs and lists them
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Set TensorFlow to use only the first GPU
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPU, 1 Logical GPU


2024-01-04 12:11:39.316383: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-01-04 12:11:39.316399: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-01-04 12:11:39.316405: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-01-04 12:11:39.316431: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-04 12:11:39.316445: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0


In [3]:
import numpy as np
from sklearn.decomposition import TruncatedSVD

# Reshape data for SVD
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Apply SVD
n_components = 50
svd = TruncatedSVD(n_components=n_components)
x_train_svd = svd.fit_transform(x_train_flat)
x_test_svd = svd.transform(x_test_flat)

# Reshape data back for CNN input
# Adjusted reshape for the SVD output
x_train_svd = x_train_svd.reshape(x_train.shape[0], n_components, 1, 1)
x_test_svd = x_test_svd.reshape(x_test.shape[0], n_components, 1, 1)


In [4]:
from tensorflow.keras.layers import Dense, Flatten, Dropout, LeakyReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.optimizers.legacy import Adam

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9)

# Model with L2 regularization and adjusted architecture
model = tf.keras.models.Sequential([
    Flatten(input_shape=(n_components, 1, 1)),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),  # L2 regularization
    Dropout(0.3),  # Increased dropout rate
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),   # L2 regularization
    Dense(10, activation='softmax')
])

# Compile the model with a lower initial learning rate
model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [5]:
# Fit the model with early stopping
model.fit(x_train_svd, y_train, epochs=10, validation_split=0.2, callbacks=[early_stopping])
model.evaluate(x_test_svd, y_test)


Epoch 1/10
   1/1500 [..............................] - ETA: 10:02 - loss: 3.1992 - accuracy: 0.0625

2024-01-04 12:11:49.768324: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-01-04 12:11:49.796653: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


1500/1500 [==============================] - 18s 12ms/step - loss: 1.3622 - accuracy: 0.6267 - val_loss: 0.6439 - val_accuracy: 0.8723
Epoch 2/10
1500/1500 [==============================] - 18s 12ms/step - loss: 0.6815 - accuracy: 0.8422 - val_loss: 0.5303 - val_accuracy: 0.8942
Epoch 3/10
1500/1500 [==============================] - 18s 12ms/step - loss: 0.5983 - accuracy: 0.8679 - val_loss: 0.4982 - val_accuracy: 0.9010
Epoch 4/10
1500/1500 [==============================] - 17s 12ms/step - loss: 0.5656 - accuracy: 0.8771 - val_loss: 0.4859 - val_accuracy: 0.9029
Epoch 5/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.5533 - accuracy: 0.8816 - val_loss: 0.4802 - val_accuracy: 0.9056
Epoch 6/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.5483 - accuracy: 0.8826 - val_loss: 0.4822 - val_accuracy: 0.9049
Epoch 7/10
1500/1500 [==============================] - 16s 11ms/step - loss: 0.5512 - accuracy: 0.8822 - val_loss: 0.4845 - val_accura

[0.4894384741783142, 0.9065999984741211]